# Laboratorio 11

La finalidad de este laboratorio es tener un mejor manejo de las herramientas que nos ofrece Scikit-Learn, como los _transformers_ y _pipelines_.  Usaremos el dataset [The Current Population Survey (CPS)](https://www.openml.org/d/534) que consiste en predecir el salario de una persona en función de atributos como la educación, experiencia o edad.

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

%matplotlib inline

Como siempre, un pequeño análisis descriptivo

In [2]:
survey = fetch_openml(data_id=534, as_frame=True)

In [3]:
X = survey.data[survey.feature_names]
X.head()

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,AGE,RACE,OCCUPATION,SECTOR,MARR
0,8.0,no,female,21.0,not_member,35.0,Hispanic,Other,Manufacturing,Married
1,9.0,no,female,42.0,not_member,57.0,White,Other,Manufacturing,Married
2,12.0,no,male,1.0,not_member,19.0,White,Other,Manufacturing,Unmarried
3,12.0,no,male,4.0,not_member,22.0,White,Other,Other,Unmarried
4,12.0,no,male,17.0,not_member,35.0,White,Other,Other,Married


In [4]:
X.describe(include="all").T.fillna("")

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
EDUCATION,534.0,,,,13.0187,2.61537,2,12,12,15,18
SOUTH,534.0,2,no,378,,,,,,,
SEX,534.0,2,male,289,,,,,,,
EXPERIENCE,534.0,,,,17.8221,12.3797,0,8,15,26,55
UNION,534.0,2,not_member,438,,,,,,,
AGE,534.0,,,,36.8333,11.7266,18,28,35,44,64
RACE,534.0,3,White,440,,,,,,,
OCCUPATION,534.0,6,Other,156,,,,,,,
SECTOR,534.0,3,Other,411,,,,,,,
MARR,534.0,2,Married,350,,,,,,,


In [5]:
y = survey.target
y.head()

0    5.10
1    4.95
2    6.67
3    4.00
4    7.50
Name: WAGE, dtype: float64

Y la posterior partición _train/test_.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

## Ejercicio 1

(1 pto)

_One-Hot Encode_ es una técnica que a partir de una _feature_ categórica generar múltiples columnas, una por categoría.

* Define el transformador `ohe_sex` utilizando `OneHotEncoder` con atributos `drop="if_binary"` y `sparse=False`, luego ajusta y transforma el dataframe `X` solo con la columna `SEX`.
* Define el transformador `ohe_race` utilizando `OneHotEncoder` con atributos `drop="if_binary"` y `sparse=False`, luego ajusta y transforma el dataframe `X` solo con la columna `RACE`.

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
ohe_sex = OneHotEncoder(drop = "if_binary", sparse = False)
ohe_sex.fit_transform(X[["SEX"]])

array([[0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],

In [9]:
ohe_race = OneHotEncoder(drop = "if_binary", sparse = False)
ohe_race.fit_transform(X[["RACE"]])

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

__Pregunta:__ ¿Por qué las transformaciones resultantes tiene diferente cantidad de columnas?

__Respuesta:__ La documentación nos dice lo siguiente

'if_binary’ : drop the first category in each feature with two categories. Features with 1 or more than 2 categories are left intact.

por lo que la razón de la diferencia de columnas es debido a las distintas categorias de cada feature. En SEX tenemos 2 categorias por lo que se elimina una columna del array sin embargo en RACE tenemos 3 categorias por lo que no se elimina ninguna columna

## Ejercicio 2

(1 pto)

Realizar _One-Hot-Encoding_ para cada una de las columnas categóricas y luego unirlas en un nuevo array o dataframe es tedioso, poco escablable y probablemente conlleve a errores. La función `make_column_transformer` permite automatizar este proceso en base a aplicar transformadores a distintas columnas.

* `categorical_columns` debe ser una lista con todos los nombres de columnas categóricas del dataframe `X`.
* `numerical_columns` debe ser una lista con todos los nombres de columnas numéricas del dataframe `X`.
* Define `preprocessor` utilizando `make_column_transformer` tal que:
    - A las columnas categóricas se les aplique `OneHotEncoder` con el argumento `drop="if_binary"`
    - El resto de las columnas se mantena igual. Hint: Revisar la documentación del argumento `remainder`.
* Finalmente define  `X_processed` al ajustar y transformar el dataframe `X` utilizando `preprocessor` 

In [10]:
X.select_dtypes(include=["float64"]).columns.values


array(['EDUCATION', 'EXPERIENCE', 'AGE'], dtype=object)

In [11]:
from sklearn.compose import make_column_transformer

categorical_columns = ['SOUTH', 'SEX', 'UNION', 'RACE', 'OCCUPATION', 'SECTOR', 'MARR']
numerical_columns = ['EDUCATION', 'EXPERIENCE', 'AGE']

preprocessor = make_column_transformer(
    (OneHotEncoder(drop = "if_binary"), categorical_columns),
    remainder="passthrough"
)
X_processed = preprocessor.fit_transform(X)
print(X_processed)

[[ 0.  0.  1. ...  8. 21. 35.]
 [ 0.  0.  1. ...  9. 42. 57.]
 [ 0.  1.  1. ... 12.  1. 19.]
 ...
 [ 0.  0.  0. ... 17. 25. 48.]
 [ 1.  1.  0. ... 12. 13. 31.]
 [ 0.  1.  1. ... 16. 33. 55.]]


In [12]:
print(f"X_processed tiene {X_processed.shape[0]} filas y {X_processed.shape[1]} columnas.")

X_processed tiene 534 filas y 19 columnas.


## Ejercicio 3

(1 pto)

Sucede un fenómeno similar al aplicar transformaciones al vector de respuesta. En ocasiones es necesario transformarlo pero que las predicciones sean en la misma escala original. `TransformedTargetRegressor` juega un rol clave, pues los insumos necesarios son: un estimador, la función y la inversa para aplicar al vector de respuesta.

Define `ttr` como un `TransformedTargetRegressor` tal que:
* El regresor sea un modelo de regresión Ridge y parámetro de regularización `1e-10`.
* La función para transformar sea logaritmo base 10. Hint: `NumPy` es tu amigo.
* La función inversa sea aplicar `10**x`. Hint: Revisa el módulo `special` de `SciPy` en la sección de _Convenience functions_.

In [13]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import Ridge
from scipy.special import exp10

In [18]:
ttr = TransformedTargetRegressor(
        regressor=Ridge(alpha=1e-10),
        func=np.log10,
        inverse_func=exp10
    )

Ajusta el modelo con los datos de entrenamiento

In [19]:
ttr.fit(X_train, y_train)

ValueError: could not convert string to float: 'no'

Lamentablemente lanza un error :(

Prueba lo siguiente:

In [20]:
ttr.fit(X_train.select_dtypes(include="number"), y_train)

TransformedTargetRegressor(func=<ufunc 'log10'>, inverse_func=<ufunc 'exp10'>,
                           regressor=Ridge(alpha=1e-10))

__Pregunta:__ ¿Por qué falló el primer ajusto? ¿Qué tiene de diferente el segundo?

__Respuesta:__ 
1. Por que se inlcuyen variables categoricas en el ajuste. 
2. Los tipos de columnas que se escojen 

## Ejercicio 4

(1 pto)

Ahora agreguemos todos los ingredientes a la juguera.

* Define `model` utilizando `make_pipeline` con los insumos `preprocessor` y `ttr`.
* Ajusta `model` con los datos de entrenamiento.
* Calcula el error absoluto medio con los datos de test.

In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import median_absolute_error

model = make_pipeline(
    preprocessor,
    ttr
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = median_absolute_error(y_test,y_pred)

print(f"El error absoluto medio obtenido es {mae}")

El error absoluto medio obtenido es 2.224855504721081
